#### LSA

In [ ]:
import pandas as pd
majors_data = pd.read_csv("./data/3. major_detail.csv")
majors_data.head(5)

In [ ]:
tokenized_doc = []

for i in range(len(majors_data)):
    keyword = majors_data['키워드'][i].split(',')
    t = ' '.join(keyword)
    tokenized_doc.append(t)
majors_data['키워드'] = tokenized_doc

In [ ]:
majors_data.head(5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
                             max_features= 1000, # 상위 1,000개의 단어를 보존 
                             max_df = 0.5, 
                             smooth_idf=True)

X = vectorizer.fit_transform(majors_data['키워드'])
X.shape # TF-IDF 행렬의 크기 확인

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=25, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

In [ ]:
import numpy as np

np.shape(svd_model.components_)

In [ ]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

lst = [[] for _ in range(25)]

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print([(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])
        lst[idx].append([(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])
        
get_topics(svd_model.components_,terms)

In [ ]:
lst

In [ ]:
lst2 = [[] for _ in range(20)]


for data in majors_data['전공명', '키워드']:
    print(data)

### LDA

In [ ]:
import pandas as pd
majors_data = pd.read_csv("./data/3. major_detail.csv")
majors_data.head(5)

In [ ]:
tokenized_doc = []

for keyword in majors_data['키워드']:
    tokenized_doc.append(keyword.split(','))
majors_data['키워드'] = tokenized_doc

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(majors_data['키워드'])
corpus = [dictionary.doc2bow(text) for text in majors_data['키워드']]
print(corpus[10])

In [ ]:
len(dictionary)

In [ ]:
import gensim
import re

NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=20)

total = []

for topic in topics:
#     print([j for i in topic[1].split("+")] for j in i.split('\" ')[1])
    
    small = []

    for i in topic[1].split("+"):
        try:
            a = i.split('*" ')[1].replace('" ', "")
        except:
            a = i.split('*"')[1].replace('" ', "")
        
        if a.find('"'):
            a = a.replace('"',"")
        small.append(a)
    
    total.append(small)
        

In [ ]:
for i in total:
    print(i)

In [ ]:
print(ldamodel.print_topics())

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)